```
01100170109835SE  DFAME Public Charter                     Alameda County Office of EducaAlameda             986    777  1020   769  918    781  776  775  105    735  93     712  78     738  728  728  3           8           10                    210    840  220    844  190    872  852  851  16     885  24     857  23     804  849  845  111    706  114    733  105    736  725  725  21     772  17     754  14     797  774  773  501    772  519    757  483    757  762  762  1           0           0                     218    717  422    707  448    735  720  721  404    748  373    722  407    753  741  742  69     629  77     627  81     629  628  628     
01100170112607SH  DEnvision Academy for Arts & Technology  Alameda County Office of EducaAlameda             200    643  221    605  226    669  639  639  123    620  104    568  107    635  608  609  2           8           4                     4           4           3                     0           1           2                     60     657  81     641  88     696  665  666  0           2           2                     8           11     569  11     672            1           9           7                     150    630  167    603  180    672  635  636  26     611  52     669  56     647  642  649  19     524  22     431  29     531  495  498     
01100170118489SE  DAspire California College Preparatory AcAlameda County Office of EducaAlameda             158    825  265    790  265    792  802  799  64     833  75     797  82     786  805  803  0           0           0                     2           2           3                     5           5           4                     87     817  181    781  174    790  796  792  0           1           0                     0           0           1                     0           1           1                     87     817  209    775  210    777  790  783  46     774  110    716  106    752  747  741  13     591  18     611  22     494  565  558     
```


In [4]:
from lxml import html as htmlparser
from os import makedirs
from os.path import basename, join, splitext
from shutil import unpack_archive
import requests
RECORD_LAYOUT_URL = 'http://www.cde.ca.gov/ta/ac/ap/reclayoutApiAvg.asp'
RAW_DATA_ZIP_URL = 'http://www3.cde.ca.gov/researchfiles/api/14avgtx.zip'
RAW_DATA_DIR = join('data', 'schools', 'raw')
RAW_DATA_ZIP_FILENAME = join(RAW_DATA_DIR, basename(RAW_DATA_ZIP_URL))
# the text file has the same name as the zip, just different extension
RAW_DATA_TXT_FILENAME = splitext(RAW_DATA_ZIP_FILENAME)[0] + '.txt'
EXTRACTED_DATA_DIR = join('data', 'schools', 'extracted')
EXTRACTED_DATA_FNAME = join(EXTRACTED_DATA_DIR, basename(RAW_DATA_TXT_FILENAME))
makedirs(RAW_DATA_DIR, exist_ok=True)
makedirs(EXTRACTED_DATA_DIR, exist_ok=True)


In [5]:
# save and extract the zip file to the raw directory
resp = requests.get(RAW_DATA_ZIP_URL)
with open(RAW_DATA_ZIP_FILENAME, 'wb') as wf:
    wf.write(resp.content)
unpack_archive(RAW_DATA_ZIP_FILENAME, extract_dir=RAW_DATA_DIR)

## Parsing the record layout

Now that we have the data downloaded and ready to unpack and parse, let's visit the [Record Layout for the 3-Year Average API Data File](http://www.cde.ca.gov/ta/ac/ap/reclayoutApiAvg.asp) get the metadata behind the fixed-width columns.

Here's a screenshot of the page:

![image cde-record-layout-page-table.png](images/schools/cde-record-layout-page-table.png)

The HTML is pretty straightforward: we need the second and fourth columns -- __Field Name__ and __Width__

```html
<table width="95%" border="1" cellpadding="3" cellspacing="0">
                <tbody><tr align="center" valign="middle">
                  <th nowrap=""><div align="center">Field 
                  #</div></th>
                  <th nowrap=""><div align="center">
                    <p>Field Name </p>
                  </div></th>
                  <th nowrap=""><div align="center">Type</div></th>
                  <th nowrap=""><div align="center">Width</div></th>
                  <th nowrap=""><div align="center">Description</div></th>
                </tr>
                <tr> 
                  <td width="6%" align="left" valign="top"> <div align="center">1 
                  </div></td>
                  <td width="13%" align="left" valign="top">CDS </td>
                  <td width="7%" align="left" valign="top">Character </td>
                  <td width="6%" align="left" valign="top"> <div align="center">14 
                  </div></td>
                  <td valign="top" width="68%">County/District/School code </td>
                </tr>
</tbody></table>
```



In [117]:
resp = requests.get(RECORD_LAYOUT_URL)
htmldoc = htmlparser.fromstring(resp.text)
_lx =  [(tr.xpath('(td[2] | td[4]/div)/text()')) for tr in htmldoc.xpath('//tr[td]')]
layouts = [[cx.strip(), int(cy)] for cx, cy in _lx]
# got to be a better way to do this...
ct = 0
for i, (k, v) in enumerate(layouts):
    layouts[i][1] = (ct, ct + v)
    ct += v

In [122]:
datarows = []
with open(RAW_DATA_TXT_FILENAME, 'r', encoding='ISO-8859-2') as rf:
    for line in rf:
        datarows.append({k: line[v[0]:v[1]].strip() for k, v in layouts})

In [123]:
datarows[0:10]

[{'AA_API11': '735',
  'AA_API12': '712',
  'AA_API13': '738',
  'AA_AVG_NW': '728',
  'AA_AVG_W': '728',
  'AA_NUM11': '105',
  'AA_NUM12': '93',
  'AA_NUM13': '78',
  'AI_API11': '',
  'AI_API12': '',
  'AI_API13': '',
  'AI_AVG_NW': '',
  'AI_AVG_W': '',
  'AI_NUM11': '3',
  'AI_NUM12': '8',
  'AI_NUM13': '10',
  'API11': '777',
  'API12': '769',
  'API13': '781',
  'AS_API11': '840',
  'AS_API12': '872',
  'AS_AVG_NW': '852',
  'AS_AVG_W': '851',
  'AS_NUM11': '210',
  'AS_NUM12': '220',
  'AS_NUM13': '190',
  'AVG_NW': '776',
  'AVG_W': '775',
  'CDS': '01100170109835',
  'CHARTER': 'D',
  'CNAME': 'Alameda',
  'DI_API11': '629',
  'DI_API12': '627',
  'DI_API13': '629',
  'DI_AVG_NW': '628',
  'DI_AVG_W': '628',
  'DI_NUM11': '69',
  'DI_NUM12': '77',
  'DI_NUM13': '81',
  'DNAME': 'Alameda County Office of Educa',
  'EL_API11': '748',
  'EL_API12': '722',
  'EL_API13': '753',
  'EL_AVG_NW': '741',
  'EL_AVG_W': '742',
  'EL_NUM11': '404',
  'EL_NUM12': '373',
  'EL_NUM13': '407'